# VeriFloat — Formal Verification of Financial Models using Z3

This interactive notebook demonstrates how to use **Z3 SMT Solver** to formally verify the consistency of simple financial models under various conditions.

We will explore:
- Basic portfolio return verification
- Impact of floating-point numerical drift
- Multi-period portfolio return consistency

In [3]:
!pip install z3-solver

## Basic Portfolio Return Verification

We model a simple portfolio return:

$$
\text{Return} = w_1 r_1 + w_2 r_2 + w_3 r_3
$$

We verify that the return remains within $[-1, 1]$ under the constraints:
- $ 0 \leq w_1, w_2, w_3 \leq 1 $
- $ w_1 + w_2 + w_3 \leq 1 $
- $ -1 \leq r_1, r_2, r_3 \leq 1 $

In [4]:
from z3 import Real, Solver, And, Or, sat

def verify_portfolio_return():
    w1, w2, w3 = Real('w1'), Real('w2'), Real('w3')
    r1, r2, r3 = Real('r1'), Real('r2'), Real('r3')

    portfolio_return = w1 * r1 + w2 * r2 + w3 * r3

    constraints = [
        w1 >= 0, w1 <= 1,
        w2 >= 0, w2 <= 1,
        w3 >= 0, w3 <= 1,
        (w1 + w2 + w3) <= 1,
        r1 >= -1, r1 <= 1,
        r2 >= -1, r2 <= 1,
        r3 >= -1, r3 <= 1
    ]

    solver = Solver()
    solver.add(constraints)
    solver.add(Or(portfolio_return < -1, portfolio_return > 1))

    result = solver.check()

    if result == sat:
        return "Violation found!", solver.model()
    else:
        return "Property holds: Portfolio return within [-1, 1]"

verify_portfolio_return()

'Property holds: Portfolio return within [-1, 1]'

## Floating-Point Drift Verification

We now introduce a small symbolic error $\epsilon$ to model the effect of **floating-point inaccuracies**.

In [5]:
def verify_portfolio_with_fp_error():
    w1, w2, w3 = Real('w1'), Real('w2'), Real('w3')
    r1, r2, r3 = Real('r1'), Real('r2'), Real('r3')
    epsilon = Real('epsilon')

    portfolio_return = w1 * r1 + w2 * r2 + w3 * r3 + epsilon

    constraints = [
        w1 >= 0, w1 <= 1,
        w2 >= 0, w2 <= 1,
        w3 >= 0, w3 <= 1,
        (w1 + w2 + w3) <= 1,
        r1 >= -1, r1 <= 1,
        r2 >= -1, r2 <= 1,
        r3 >= -1, r3 <= 1,
        epsilon >= -0.001, epsilon <= 0.001
    ]

    solver = Solver()
    solver.add(constraints)
    solver.add(Or(portfolio_return < -1, portfolio_return > 1))

    result = solver.check()

    if result == sat:
        return "Violation with numerical drift!", solver.model()
    else:
        return "Property holds even under small numerical errors."

verify_portfolio_with_fp_error()

('Violation with numerical drift!',
 [r2 = 65535/65536,
  r3 = 0,
  epsilon = 65/65536,
  r1 = 0,
  w1 = 0,
  w2 = 1023/1024,
  w3 = 0])

## Multi-Period Portfolio Return Verification

We extend the model to **two periods** and check whether the combined return over two periods stays within safe bounds.

In [6]:
def verify_two_period_portfolio():
    w1, w2, w3 = Real('w1'), Real('w2'), Real('w3')
    r1_1, r2_1, r3_1 = Real('r1_1'), Real('r2_1'), Real('r3_1')
    r1_2, r2_2, r3_2 = Real('r1_2'), Real('r2_2'), Real('r3_2')

    constraints = [
        w1 >= 0, w1 <= 1,
        w2 >= 0, w2 <= 1,
        w3 >= 0, w3 <= 1,
        (w1 + w2 + w3) <= 1,
    ] + [And(r >= -1, r <= 1) for r in [r1_1, r2_1, r3_1, r1_2, r2_2, r3_2]]

    period1_return = w1 * r1_1 + w2 * r2_1 + w3 * r3_1
    period2_return = w1 * r1_2 + w2 * r2_2 + w3 * r3_2

    total_return = period1_return + period2_return

    solver = Solver()
    solver.add(constraints)
    solver.add(Or(total_return < -2, total_return > 2))

    result = solver.check()

    if result == sat:
        return "Violation found in multi-period case!", solver.model()
    else:
        return "Property holds in multi-period scenario."

verify_two_period_portfolio()

'Property holds in multi-period scenario.'

## Conclusion

**VeriFloat** demonstrates how formal methods can ensure numerical correctness in financial models, even under uncertainty and floating-point drift.

This foundation can be extended to more complex financial algorithms for risk analysis, automated reasoning, and verification of safety-critical financial systems.